In [11]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

import re
import os
import pickle
from enum import Enum

In [ ]:
"""
Oauth consent https://console.cloud.google.com/apis/credentials/consent?project=potent-ripple-377913
youtube = YouTube(credentials_file='', token_path='')
youtube.authenticate()
youtube.search_video(): query -> List[YouTubeVideo]
youtube.search_channel(): query -> List[YouTubeChannel]
youtube.search_playlist(): query -> List[PlayList]
youtube.find_video_by_id(): video_id -> YouTubeVideo
youtube.find_video_by_url(): video_url -> YouTubeVideo
youtube.utils
"""

In [2]:
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [3]:
class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    __TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIConstants.API_VERSION
    __SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.__credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.__api_token_path = self.__get_default_api_token_path()
        else:
            self.__api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.__api_token_path):
            with open(self.__api_token_path, "rb") as token:
                self.__credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            # save the credentials for the next run
            with open(self.__api_token_path, "wb") as token:
                pickle.dump(self.__credentials, token)

        return build(self.__API_SERVICE_NAME, self.__API_VERSION, credentials=self.__credentials)
    
    def authenticate(self):
        try:
            youtube_client =  self.__authenticate_youtube()
        except RefreshError as e:
            #delete token
            raise Exception('The token is expired. Kindly generate a new one.')
        else:
            return youtube_client

In [4]:
class YouTubeSearchQuery:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValueError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

        
class YouTubeVideoSearchQuery(YouTubeSearchQuery):
    pass

In [5]:
class YouTubeSearchType:
    VIDEO = 'video'
    CHANNEL = 'channel'
    PLAYLIST = 'playlist'

In [15]:
class YouTubeSearch:
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, *args, **kwargs):
        pass

In [102]:
class YouTubeVideoStats:
    def __init__(self, viewCount: int, likeCount: int, commentCount: int):
        self.__view_count = int(viewCount)
        self.__like_count = int(likeCount)
        self.__comment_count = int(commentCount)
        
    def get_video_stats(self):
        video_stats = {
            'view_count': self.__view_count,
            'like_count': self.__like_count,
            'comment_count': self.__comment_count
        }
        return video_stats

In [103]:
class YouTubeVideoDetails:
    def __init__(self, id: str, channelId: str, title: str, channelTitle: str, 
                 description: str, tags: list[str], duration: str, licensedContent: bool):
        self.__id = id
        self.__channel_id = channelId
        self.__title = title
        self.__channel_title = channelTitle
        self.__description = description
        self.__tags = tags
        self.__duration = duration
        self.__licensed_content = licensedContent
        
    def get_video_details(self):
        video_details = {
            'id': self.__id,
            'channel_id': self.__channel_id,
            'title': self.__title,
            'channel_title': self.__channel_title,
            'description': self.__description,
            'tags': self.__tags,
            'duration': self.__duration,
            'licensed_content': self.__licensed_content
        }
        return video_details
    
    def get_video_id(self):
        return self.__id

In [104]:
class YouTubeVideo:
    """A YouTube Video."""
    def __init__(self, video_details):
        self.__video_stats = self.__create_video_stats(video_details)
        self.__video_details = self.__create_video_details(video_details)
        self.__video_top_level_comments = None
        
    def get_video_stats_details(self):
        video_stats_details = dict()
        video_stats_details['details'] = self.get_video_details()
        video_stats_details['statistics'] = self.get_video_stats()
        return video_stats_details
    
    def get_video_comments(self, youtube_client):
        if not self.__video_top_level_comments:
            youtube_commenthread = YouTubeCommentThread(self.get_video_id())
            self.__video_top_level_comments = youtube_commenthread.get_video_comments(youtube_client)
        return self.__video_top_level_comments
        
    def __create_video_stats(self, video_details: dict):
        video_stats = YouTubeVideoStats(**video_details['statistics'])
        return video_stats
    
    def __create_video_details(self, video_details: dict):
        video_details = YouTubeVideoDetails(**video_details['details'])
        return video_details
        
    def get_video_stats(self):
        return self.__video_stats.get_video_stats()
    
    def get_video_details(self):
        return self.__video_details.get_video_details()
    
    def get_video_top_level_comments(self):
        pass
    
    def get_video_id(self):
        return self.__video_details.get_video_id()
    
    def get_video_title(self):
        pass

In [105]:
class VideoSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
            regionCode=self.__REGION_CODE
        ) 
        return basic_info_params
    
    def search_video(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response(search_response, youtube_client)
        return videos
    
    def __parse_basic_response(self, search_response, youtube_client):
        videos = []
        video_results = search_response['items']
        for video_result in video_results:
            video_id = video_result['id']['videoId']
            youtube_video = FindVideo(video_id).find_video(youtube_client)
            videos.append(youtube_video)
        return videos

In [106]:
class FindVideo:
    def __init__(self, video_id: str):
        """Find the video with the given id."""
        self.__video_id = video_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        youtube_video = YouTubeVideo(parsed_response)
        return youtube_video
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        if items['snippet'].get('tags'):
            parsed_video_details['details']['tags'] = items['snippet']['tags']
        else:
            parsed_video_details['details']['tags'] = []
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [107]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = self.__youtube_client = self.__auth.authenticate()

    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        videos = VideoSearch(query_string).search_video(self.__youtube_client)
        return videos
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [44]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube_client = youtube.get_youtube()

In [45]:
videos = youtube.search_video('python programming')

In [47]:
videos[0].get_video_stats_details()

{'details': {'id': 'kqtD5dpn9C8',
  'channel_id': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'channel_title': 'Programming with Mosh',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get started! \n👍 Subscribe for more Python tutorials like this: https://goo.gl/6PYaGF\n🔥 Want to learn more? Watch my complete Python course: https://youtu.be/_uQrJ0TkZlc\n\n📕 Get my FREE Python cheat sheet: http://bit.ly/2Gp80s6\n\nCourses: https://codewithmosh.com\nTwitter: https://twitter.com/moshhamedani\nFacebook: https://www.facebook.com/programmingwithmosh/\nBlog: http://programmingwithmosh.com\n\n#Python, #MachineLearning, #WebDevelopment\n\n📔 Python Exercises for Beginners: https://goo.gl/1XnQB1\n\n⭐ My Favorite Python Books\n- Python Crash Course: https://amzn.to/2GqMdjG\n- Automate the Boring Stuff with Python: https://amzn.to/2N71d6S\n- A Smarter Way to L

In [131]:
class YouTubeComment:
    def __init__(self, id: str, videoId: str, totalReplyCount: str, textDisplay: str, 
                authorDisplayName: str, authorProfileImageUrl: str, authorChannelId: str,
                likeCount: str, publishedAt: str, updatedAt: str):
        self.__id = id
        self.__video_id = videoId
        self.__total_reply_count = totalReplyCount
        self.__text_display = textDisplay
        self.__author_display_name = authorDisplayName
        self.__author_profile_image_url = authorProfileImageUrl
        self.__author_channel_id = authorChannelId
        self.__like_count = int(likeCount)
        self.__published_at = publishedAt
        self.__updated_at = updatedAt
        
    def get_comment(self):
        comment = dict(
            id=self.__id,
            videoId=self.__video_id,
            totalReplyCount=self.__total_reply_count,
            textDisplay=self.__text_display,
            authorDisplayName = self.__author_display_name,
            authorProfileImageUrl = self.__author_profile_image_url,
            authorChannelId = self.__author_channel_id,
            likeCount = self.__like_count,
            publishedAt = self.__published_at,
            updatedAt = self.__updated_at
        )
        return comment
    
    def get_comment_text(self):
        comment_text = self.__text_display
        return comment_text
    
    def __str__(self):
        return self.get_comment_text()
    
    def __repr__(self):
        return f"YouTubeComment(id='{self.__id}', videoId='{self.__video_id}', \
        totalReplyCount={self.__total_reply_count})"

In [116]:
class YouTubeCommentThread:
    def __init__(self, video_id: str):
        self.__video_id = video_id
        
    def get_video_comments(self, youtube_client):
        """Get the top level comments for a video."""
        youtube_comments = self.__find_comments(youtube_client)
        youtube_comments = [self.__create_comment(comment) for comment in youtube_comments]
        return youtube_comments
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            videoId=self.__video_id,
            part='snippet,replies'
        ) 
        return basic_info_params
    
    def __find_comments(self, youtube_client):
        """Find the video comments."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.commentThreads().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        comments = self.__parse_comments(search_response)
        return comments
    
    def __create_comment(self, comment_details):
        youtube_comment = YouTubeComment(**comment_details)
        return youtube_comment

    def __parse_comments(self, search_response):
        items = search_response['items']
        comments = []
        for item in items:
            comments.append({
                'id': item['id'],
                'videoId': item['snippet']['videoId'],
                'totalReplyCount': item['snippet']['totalReplyCount'],
                'textDisplay': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                'authorDisplayName': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'authorProfileImageUrl': item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl'],
                'authorChannelId': item['snippet']['topLevelComment']['snippet']['authorChannelId']['value'],
                'likeCount': item['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': item['snippet']['topLevelComment']['snippet']['updatedAt']
        })
        return comments

In [66]:
youtube_commenthread = YouTubeCommentThread('kqtD5dpn9C8')
youtube_comments = youtube_commenthread.get_video_comments(youtube_client)

In [67]:
youtube_comments

[{'id': 'Ugzby30dYHdobijJxnF4AaABAg',
  'videoId': 'kqtD5dpn9C8',
  'totalReplyCount': 46,
  'textDisplay': '🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>',
  'authorDisplayName': 'Programming with Mosh',
  'authorProfileImageUrl': 'https://yt3.ggpht.com/tBEPr-zTNXEeae7VZKSZYfiy6azzs9OHowq5ZvogJeHoVtKtEw2PXSwzMBKVR7W0MI7gyND8=s48-c-k-c0x00ffffff-no-rj',
  'authorChannelId': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
  'likeCount': 224,
  'publishedAt': '2022-03-07T23:14:30Z',
  'updatedAt': '2022-09-20T17:50:56Z'},
 {'id': 'UgxwxwZcrMaDg7t4r-J4AaABAg',
  'videoId': 'kqtD5dpn9C8',
  'totalReplyCount': 0,
  'textDisplay': 'weight = float(input (&quot;Weight: &quot;))\r<br>KG = input(&quot;(K)g or (L)bs: &quot;)\r<br>if KG.upper() == &quot;K&quot;:\r<br>    print(&quot;Weight in Kg: &quot;, weight * 0.45)\r<br>elif KG == &quot;L

In [79]:
yt_comment = YouTubeComment(**youtube_comments[0])

In [80]:
yt_comment.get_comment()

{'id': 'Ugzby30dYHdobijJxnF4AaABAg',
 'videoId': 'kqtD5dpn9C8',
 'totalReplyCount': 46,
 'textDisplay': '🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>',
 'authorDisplayName': 'Programming with Mosh',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/tBEPr-zTNXEeae7VZKSZYfiy6azzs9OHowq5ZvogJeHoVtKtEw2PXSwzMBKVR7W0MI7gyND8=s48-c-k-c0x00ffffff-no-rj',
 'authorChannelId': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
 'likeCount': 224,
 'publishedAt': '2022-03-07T23:14:30Z',
 'updatedAt': '2022-09-20T17:50:56Z'}

In [81]:
yt_comment.get_comment_text()

'🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>'

In [83]:
youtube_commenthread = YouTubeCommentThread('kqtD5dpn9C8')
youtube_comments = youtube_commenthread.get_video_comments(youtube_client)

In [84]:
youtube_comments

In [85]:
youtube_comments[0].get_comment_text()

'🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>'

In [90]:
videos = youtube.search_video('python programming')

In [91]:
videos

In [93]:
videos[0].get_video_comments(youtube_client)

In [98]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube_client = youtube.get_youtube()

In [110]:
videos = youtube.search_video('python programming')

In [111]:
videos[0].get_video_comments(youtube_client)

In [112]:
str(videos[0].get_video_comments(youtube_client))

'[<__main__.YouTubeComment object at 0x7f3a7de3cac0>, <__main__.YouTubeComment object at 0x7f3a7d9e2200>, <__main__.YouTubeComment object at 0x7f3a7d3590c0>, <__main__.YouTubeComment object at 0x7f3a7d35a3e0>, <__main__.YouTubeComment object at 0x7f3a7d359510>, <__main__.YouTubeComment object at 0x7f3a7d358ac0>, <__main__.YouTubeComment object at 0x7f3a7d35a2f0>, <__main__.YouTubeComment object at 0x7f3a7d35ba90>, <__main__.YouTubeComment object at 0x7f3a7d3591b0>, <__main__.YouTubeComment object at 0x7f3a7d35b700>, <__main__.YouTubeComment object at 0x7f3a7d35b670>, <__main__.YouTubeComment object at 0x7f3a7d35a4d0>, <__main__.YouTubeComment object at 0x7f3a7d3c7a30>, <__main__.YouTubeComment object at 0x7f3a7d3c7b50>, <__main__.YouTubeComment object at 0x7f3a7d3c7e20>, <__main__.YouTubeComment object at 0x7f3a7d3c6cb0>, <__main__.YouTubeComment object at 0x7f3a7d3c7340>, <__main__.YouTubeComment object at 0x7f3a7d3c70a0>, <__main__.YouTubeComment object at 0x7f3a7d3c7370>, <__main__.

In [130]:
youtube_commenthread = YouTubeCommentThread('kqtD5dpn9C8')
youtube_comments = youtube_commenthread.get_video_comments(youtube_client)
youtube_comments

[YouTubeComment(id='Ugzby30dYHdobijJxnF4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=46),
 YouTubeComment(id='UgxwxwZcrMaDg7t4r-J4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgzVmRNNr6yCSctO2DZ4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='Ugwnb4OHB2u1Apczep14AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgykWB84sBHOdPhSGVV4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='Ugy9ePbQKDs3WEcfQjN4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgwQmf-KDnOvx5KvNSl4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgzmqcAEzFqZLEVycox4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgxLQwzb1E6lksW2wAl4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeComment(id='UgzwH17h6Wkj3Gawayl4AaABAg', videoId='kqtD5dpn9C8',        totalReplyCount=0),
 YouTubeC

In [123]:
str(youtube_comments[0])

'🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>'

In [124]:
print(youtube_comments[0])

🔥 Want to master Python? Get my complete Python course: <a href="http://bit.ly/35BLHHP">http://bit.ly/35BLHHP</a><br>👍 Subscribe for more Python tutorials like this: <a href="https://goo.gl/6PYaGF">https://goo.gl/6PYaGF</a>
